Generate derivative structures (Phys. Rev. B **77**, 224115 (2008)). Take $P6/mmm$ lattice as example.

In [5]:
using Random
using LinearAlgebra
using SmithNormalForm
using NPZ
using JLD
using Combinatorics
using Statistics

Generate **Hermite Normal Form (HNF)** matrix by given dimensionality and determinant.

This step aims for superlattice deduction based on fixed parent lattice.

In [228]:
function diag_single(det)
    factor_list = []
    for det_ in det
        for i in 1:det_ 
            if mod(det_, i) == 0
                append!(factor_list, i)
            end
        end 
    end
    return Int64.(vec(factor_list))
end

det = 5 #* determinant for Hermite normal form matrix
dim = 3

factor_set = diag_single(det)
hnf_list = Dict()
hnf_list[1] = factor_set
global new_p, new_prod, prod_p, fac_p, fac_p_fac, p_
for i in 2:dim
    prod_ = hnf_list[i-1]
    new_prod = []
    for p_ in prod_ 
        prod_p = prod(p_)
        if mod(det, prod_p) != 0
            continue
        end
        fac_p = div(det, prod_p)
        for fac_p_fac in diag_single(fac_p)
            new_p = Int64.(reduce(vcat, [p_, fac_p_fac]))
            # println(new_p)
            push!(new_prod, new_p)
        end
    end 
    hnf_list[i] = new_prod

    #* Filter the factor set which meets the requirement of determinant
    if i == dim
        hnf_list[i] = filter(x -> det == prod(x), hnf_list[i])
        hnf_list[i] = unique(hcat(hnf_list[i]...), dims=2)
    end
end

#* Use 3d space as an example
mat_buffer = []
smith_buffer = [] 
global diag_, b_range, d_e_range, prod_de, mat
for i in 1:size(hnf_list[dim], 2)
    diag_ = hnf_list[dim][:, i]
    b_range = 0:diag_[2]-1
    d_e_range = 0:diag_[3]-1
    prod_de = collect(Iterators.product(d_e_range, d_e_range))
    # println(prod_de)
    for b_ in b_range
        mat = diagm(diag_)
        mat[2,1] = b_
        for de_ in prod_de
            mat_ = copy(mat)
            mat_[3,1] = de_[1]
            mat_[3,2] = de_[2]
            push!(mat_buffer, mat_)
            push!(smith_buffer, diagm(smith(mat_)))
        end
    end 
end

In [ ]:
hcp_lattice = Matrix{Float64}([
    1 1/2 0; 0 sqrt(3)/2 0; 0 0 3.3/3.1])

displacement_set = Matrix{Float64}([
    0 0 0; 1/3 sqrt(3)/6+1/6 1/2*sqrt(8/3); 2/3 sqrt(3)/3+1/3 1/2*sqrt(8/3)])'

#* n=1, the original AlB2 lattice
basis_n1 = hcp_lattice

#* n=2, the basis is AH where H is the hermite normal form 
snf_n2 = smith(mat_buffer[1])
hcp_basis_buffer = []
for i in 1:length(mat_buffer)
    h_ = mat_buffer[i]
    hcp_basis = hcp_lattice * h_
    push!(hcp_basis_buffer, hcp_basis)
end 

"""
rotation matrix for hexagonal lattice
"""
h_rotation_buffer = [
    [-1/2 sqrt(3)/2 0; -sqrt(3)/2 -1/2 0; 0 0 1],
    [-1/2 -sqrt(3)/2 0; sqrt(3)/2 -1/2 0; 0 0 1],
    [1/2 sqrt(3)/2 0; -sqrt(3)/2 1/2 0; 0 0 1],
    [1/2 -sqrt(3)/2 0; sqrt(3)/2 1/2 0; 0 0 1],
    [-1/2 -sqrt(3)/2 0; -sqrt(3)/2 1/2 0; 0 0 1],
    [-1/2 sqrt(3)/2 0; sqrt(3)/2 1/2 0; 0 0 1],
    [1/2 sqrt(3)/2 0; sqrt(3)/2 -1/2 0; 0 0 1],
    [1/2 -sqrt(3)/2 0; -sqrt(3)/2 -1/2 0; 0 0 1],    
]

println("length of hcp basis matrices is $(length(hcp_basis_buffer))")

length of hcp basis matrices is 31


In [230]:
hcp_basis = Matrix{Float64}[]
sym_identical_buffer = []

for n_base in 1:length(hcp_basis_buffer)
    h1_n2 = hcp_basis_buffer[n_base]

    any_in_buffer = false
    for j in 2:length(hcp_basis_buffer)
        for i in 1:length(h_rotation_buffer)
            basis_h = hcp_basis_buffer[j]
            h_ = h_rotation_buffer[i, :][1]
            hcp_basis = round.(inv(h1_n2) * inv(h_) * basis_h,
                            digits=2)
            if all(isinteger.(hcp_basis))
                any_in_buffer = true
                if length(sym_identical_buffer) == 0
                    push!(sym_identical_buffer, [n_base, j])
                else 
                    in_buffer = false
                    for k in 1:length(sym_identical_buffer)
                        sym_buffer = sym_identical_buffer[k]
                        if n_base ∈ sym_buffer || j ∈ sym_buffer
                            append!(sym_buffer, [n_base, j])
                            sym_identical_buffer[k] = unique(sym_buffer)
                            in_buffer = true
                            break
                        end
                    end 
                    if !in_buffer
                        push!(sym_identical_buffer, [n_base, j])
                    end
                end 
            end              
            # println(basis_h, j, mat_buffer[j])
        end
    end
    if !any_in_buffer
        push!(sym_identical_buffer, [n_base])
    end
end

println(sym_identical_buffer, "\n length of sym_identical_buffer is $(length(sym_identical_buffer))")

Any[[1], [2, 5, 6, 7, 21, 25], [3, 4, 11, 13, 16, 19], [8, 10, 12, 20, 22, 24], [9, 14, 15, 17, 18, 23], [26, 30, 31], [27, 28, 29]]
 length of sym_identical_buffer is 7


In [231]:
hcp_chosen_dict = Dict()
hnf_dict = Dict()
count = 1
vec_distinct = [vec[1] for vec in sym_identical_buffer]
for i in vec_distinct
    hcp_chosen_dict[count] = hcp_basis_buffer[i]
    hnf_dict[count] = mat_buffer[i]
    count += 1
end

hcp_chosen_dict, hnf_dict

(Dict{Any, Any}(5 => [1.0 0.5 0.0; 0.0 0.8660254037844386 0.0; 3.193548387096774 1.064516129032258 5.32258064516129], 4 => [1.0 0.5 0.0; 0.0 0.8660254037844386 0.0; 2.129032258064516 1.064516129032258 5.32258064516129], 6 => [1.0 2.5 0.0; 0.0 4.330127018922193 0.0; 0.0 0.0 1.064516129032258], 7 => [1.5 2.5 0.0; 0.8660254037844386 4.330127018922193 0.0; 0.0 0.0 1.064516129032258], 2 => [1.0 0.5 0.0; 0.0 0.8660254037844386 0.0; 1.064516129032258 0.0 5.32258064516129], 3 => [1.0 0.5 0.0; 0.0 0.8660254037844386 0.0; 2.129032258064516 0.0 5.32258064516129], 1 => [1.0 0.5 0.0; 0.0 0.8660254037844386 0.0; 0.0 0.0 5.32258064516129]), Dict{Any, Any}(5 => [1 0 0; 0 1 0; 3 1 5], 4 => [1 0 0; 0 1 0; 2 1 5], 6 => [1 0 0; 0 5 0; 0 0 1], 7 => [1 0 0; 1 5 0; 0 0 1], 2 => [1 0 0; 0 1 0; 1 0 5], 3 => [1 0 0; 0 1 0; 2 0 5], 1 => [1 0 0; 0 1 0; 0 0 5]))

In [232]:
pth_sav_hcpbasis = "runs/hnf_$(det)_hcp/hcp_basis.jld"
pth_sav_hnf = "runs/hnf_$(det)_hcp/hnf.jld"
save(pth_sav_hcpbasis, "hcp_basis", hcp_chosen_dict)
save(pth_sav_hnf, "hnf", hnf_dict)

In [ ]:
a = 3.1
c = 3.3
specie_denote_list = ["TiZrHfB"]

for specie_denote in specie_denote_list
    hcp_lattice = Matrix{Float64}([
        1 1/2 0; 0 sqrt(3)/2 0; 0 0 c/a])

    function c_cols(itr)
        n = length(first(itr))
        reshape([xj[i] for i in 1:n, xj in itr], n, :)
    end

    f_range = 16
    f_raw = c_cols(collect(Iterators.product(-f_range:f_range,
                                            -f_range:f_range,
                                            -f_range:f_range)))
    f_d1 = copy(f_raw) .+ Vector{Float64}([1/3, 1/3, 1/2])
    f_d2 = copy(f_raw) .+ Vector{Float64}([2/3, 2/3, 1/2])

    det = 3 #TODO HNF det. for the basis
    pth_sav_hcpbasis = "runs/hnf_$(det)_hcp/hcp_basis.jld"
    pth_sav_hnf = "runs/hnf_$(det)_hcp/hnf.jld"
    hcp_chosen_dict = load(pth_sav_hcpbasis)["hcp_basis"]
    hnf_dict = load(pth_sav_hnf)["hnf"]

    fraw_fn_rn = Dict()
    for hnf_i in 1:length(hnf_dict)
        hnf_test = hnf_dict[hnf_i]
        f_n = Vector{Float64}[]
        valid_f = []
        
        for i in 1:size(f_raw, 2)

            f_ = f_raw[:, i]
            f_d1_ = f_d1[:, i]
            f_d2_ = f_d2[:, i]

            f_n = inv(hnf_test) * f_
            f_n_d1, f_n_d2 = inv(hnf_test) * f_d1_, inv(hnf_test) * f_d2_
            if sum(all.(-1e-2 .<= f_n .< 1-1e-2)) == 3
                push!(valid_f, [f_, round.(f_n, digits=64), 
                    hcp_lattice*hnf_test*f_n*a, round.(hcp_lattice*hnf_test*a, digits=64), "TM"])
            end
            if sum(all.(-1e-2 .<= f_n_d1 .< 1-1e-2)) == 3 #* To check if fractional coordinates within [0, 1)
                push!(valid_f, [f_d1_, round.(f_n_d1, digits=64), 
                    hcp_lattice*hnf_test*f_n_d1*a, round.(hcp_lattice*hnf_test*a, digits=64), "B"])
            end
            if sum(all.(-1e-2 .<= f_n_d2 .< 1-1e-2)) == 3
                push!(valid_f, 
                [f_d2_, round.(f_n_d2, digits=64), 
                    hcp_lattice*hnf_test*f_n_d2*a, round.(hcp_lattice*hnf_test*a, digits=64), "B"])
            end
        end
        fraw_fn_rn[hnf_i] = valid_f
    end

    pth_sav_lattice = "runs/hnf_$(det)_hcp/fraw_fn_rn_$(specie_denote).jld"
    save(pth_sav_lattice, "fraw_fn_rn", fraw_fn_rn)
    println("$(specie_denote) is done")
end